In [1]:
# Input the complete matrix, not just lower triangular

coxeter_matrix = matrix([[1,3,2], [3,1,6],[2,6,1]])

dim_v = coxeter_matrix.rank()
# Vector space over the reals
v = VectorSpace(RR,dim_v)

In [2]:
coxeter_matrix

[1 3 2]
[3 1 6]
[2 6 1]

In [8]:
def bilinear(x,y):
    '''Computes the bilinear form of two roots'''
    temp = 0
    for i in range(dim_v):
        for j in range(dim_v):
            temp += x[i]*y[j]*(-cos(pi/coxeter_matrix[i,j])) 
    return temp

def s(i):
    '''Returns the ith simple root'''
    return v.basis()[i]

def simple_reflection(s,x):
    return x-2*bilinear(x,s)*s

def w_action(word_list,x):
    '''Input a word w in the generators as a list and a root \alpha. Returns w(\alpha)'''
    temp_x = x
    word_list.reverse()
    
    for s in word_list:
        temp_x = simple_reflection(s,temp_x)
    
    # reverse back is it reverses the word outside this function as well
    word_list.reverse()
    
    return temp_x

def inversion_set(w):
    '''Returns the left inversion set of an element w as a list'''
    inversionlist = [w[0]]

    for i in range(len(w)-1):        
        # note: w[0:i+1] does not include the endpoint i+1
        temp = w[0:i+1]
        inversionlist.append(w_action(temp,w[i+1]))
        
    return inversionlist

def elementary_roots():
    '''Returns the set of elementary roots'''
    elementary = []
    
    for i in range(dim_v):
        elementary.append(s(i))
    
    temp_n = 1
    counter = 0
    
    while temp_n < len(elementary):
        
        new_root_counter = 0
        
        temp_n = len(elementary)

        for root in elementary[counter:]:
            for i in range(dim_v):

                temp = w_action([s(i)],root)

                if temp[i] > root[i] and temp[i]-root[i] < 2 and temp not in elementary:
                    elementary.append(temp)
                    new_root_counter += 1
        
        counter = len(elementary) - new_root_counter - 1
                      
    return elementary

def elementary_inversion_set(w, elementary):
    '''returns the left elementary inversion set of an element, but compute the elementary roots first and input it, 
       to avoid recalculating it everytime.'''
    
    e_w = inversion_set(w)
    e_w_copy = e_w[:] 
    
    for root in e_w_copy:
        if root not in elementary:
            e_w.remove(root)
            
    return e_w

def left_length_increase(word, s):
    '''Tells you whether a word increases in length by multiplication by s on the left'''
    
    if word[0] == s:
        return False
    
    word_copy = word[:]
    word_copy.reverse()
    
    root = w_action(word_copy,s)
    
    if sum(root) >= 0:
        return True
    else:
        return False
    
def words_are_same_element(word1, word2):
    '''Given two reduced words, determine whether they are the same element 
    by checking if their inversion sets are the same'''
    
    # this only works because we assume words are reduced
    if len(word1) != len(word2):
        return False
  
    w1_inversion = inversion_set(word1)
    w2_inversion = inversion_set(word2)
        
    for root in w1_inversion:
        if root in w2_inversion:
            w2_inversion.remove(root)
    
    if len(w2_inversion) == 0:
        return True
    else:
        return False

def word_nice(word):
    '''prints word as a string of letters'''
    word_string = ''
    for vector in word:
        if vector == s(0):
            word_string += 's'
        elif vector == s(1):
            word_string += 't'
        elif vector == s(2):
            word_string += 'u'
        else:
            word_string += 'v'
    
    return word_string

def elementary_inversion_lists_same(e1, e2):
    '''given two elementary inversion lists, check whether they are the same as sets'''
    if len(e1) != len(e2):
        return False
    
    e1_copy = e1[:]
    
    for root in e1:
        if root in e2:
            e1_copy.remove(root)
        else:
            return False
    
    if len(e1_copy) == 0:
        return True
    else:
        return False

In [9]:
elementary = elementary_roots()
len(elementary)

12

## Some code for dihedral root subsystems

In [4]:
import math

def dihedral_root_system(r1, r2):
    '''Returns the root system for the dihedral group generated by the reflections r1 and r2
       input is two reflections as a list
    '''
    
    # Get root associated to reflection 1
    middle_of_r1 = int(math.ceil(len(r1)/2))
    
    if middle_of_r1 - 2==0:
        root1 = w_action([r1[0]],r1[middle_of_r1 - 1])
    elif middle_of_r1 == 1:
        root1 = r1[0]
    else:
        root1 = w_action(r1[0:middle_of_r1 - 1],r1[middle_of_r1 - 1])
    
    # Get root associated to reflection 2
    middle_of_r2 = int(math.ceil(len(r2)/2))
    
    if middle_of_r2 - 2==0:
        root2 = w_action([r2[0]],r2[middle_of_r2 - 1])
    elif middle_of_r2 == 1:
        root2 = r2[0]
    else:
        root2 = w_action(r2[0:middle_of_r2 - 1],r2[middle_of_r2 - 1])
    
    roots = [root1, root2]
    reflections = [r1, r2]
    
    temp_n = 1
    counter = 0
    
    while temp_n < len(roots):
        
        new_root_counter = 0
        
        temp_n = len(roots)
        
        if temp_n > 200:
            print("Looks like this will be an infinite dihedral group. I'm going to stop!!")
            break
        
        for root in roots[counter:]:
            for re in reflections:
                new_root = w_action(re,root)
                
                if new_root not in roots:
                    roots.append(new_root)
                    new_root_counter += 1
        
        counter = len(roots) - new_root_counter - 1
        
    return roots

def positive_roots(subgroup):
    
    pos_roots = []
    
    for el in subgroup:
        if sum(el) > 0:
            pos_roots.append(el)
            
    return pos_roots

In [6]:
w_action([s(0),s(2),s(1)], s(2))

(1.00000000000000*sqrt(3), 1.00000000000000*sqrt(3), 2.00000000000000)

## Construct the Brink-Howlett automata

In [10]:
def bh_automata():
    '''Returns the list of states, where each item is a list with first index a state
    and then following items in the list are tuples with a transition letter and target state'''

    # list of states and their transition information
    # as elementary inversion lists
    states_transitions = []
    
    # list of states only (as representative word)
    state_list = [[s(0)], [s(1)], [s(2)]]
    
    counter = 0
    num_of_new_states = 1
    
    while num_of_new_states > 0:
        
        num_of_new_states = 0
        
        for state in state_list[counter:]:
            
            counter = len(state_list)-num_of_new_states
            
            state_elementary = elementary_inversion_set(state, elementary)
            state_info = [state_elementary]
            
            for j in range(0,dim(v)):
                print(' ')
                print('Word representing inversion set is: ' + word_nice(state) + ' generator is: ' + word_nice([s(j)]) )
                
                if left_length_increase(state,s(j)):

                    print('Yes, ' + word_nice(state) + ' increases on the left by ' + word_nice([s(j)]))
                    
                    target_state = [s(j)] + state
                    target_state_word = word_nice(target_state)
                    
                    print('Word repping Target State: ' + target_state_word)
                    
                    target_state_elementary_inversion_set = elementary_inversion_set(target_state, elementary)
                                        
                    transition = (s(j), target_state_elementary_inversion_set)
                    state_info.append(transition)

                    # If target state elementary inversion set is not in the list of states, add to the list.
                    target_state_is_new = True
                    
                    for st in state_list:
                        st_elementary = elementary_inversion_set(st, elementary)
                        
                        if elementary_inversion_lists_same(target_state_elementary_inversion_set, st_elementary):
                            print('Its inversion set is already represented in state_list')
                            
                            target_state_is_new = False
                            break
                    
                    print('Status of targ_state_is_new: ')
                    print(target_state_is_new)
                    
                    if target_state_is_new == True:
                        # add target_state rep to state list
                        state_list.append(target_state)
                        print('New State. Adding to state_list, the rep: ' + word_nice(target_state))
                        num_of_new_states += 1

                else:
                    print('No, ' + word_nice(state) + ' decreases by ' + word_nice([s(j)]))
                    
            states_transitions.append(state_info)
        
         
        print('num_of_new_states: ' + str(num_of_new_states))
        print('NUMBER OF STATES IS NOW: ' + str(len(state_list)))
        print('State reps are: ')
        print(list(map(word_nice, state_list)))
        print('The COUNTER is at: ' + str(counter))
        
    print('Number of states: ' + str(len(state_list)))
    print(list(map(word_nice, state_list)))
    return states_transitions
            

In [11]:
test = bh_automata()

 
Word representing inversion set is: s generator is: s
No, s decreases by s
 
Word representing inversion set is: s generator is: t
Yes, s increases on the left by t
Word repping Target State: ts
Status of targ_state_is_new: 
True
New State. Adding to state_list, the rep: ts
 
Word representing inversion set is: s generator is: u
Yes, s increases on the left by u
Word repping Target State: us
Status of targ_state_is_new: 
True
New State. Adding to state_list, the rep: us
 
Word representing inversion set is: t generator is: s
Yes, t increases on the left by s
Word repping Target State: st
Status of targ_state_is_new: 
True
New State. Adding to state_list, the rep: st
 
Word representing inversion set is: t generator is: t
No, t decreases by t
 
Word representing inversion set is: t generator is: u
Yes, t increases on the left by u
Word repping Target State: ut
Status of targ_state_is_new: 
True
New State. Adding to state_list, the rep: ut
 
Word representing inversion set is: u genera

Status of targ_state_is_new: 
True
New State. Adding to state_list, the rep: ustus
 
Word representing inversion set is: utus generator is: s
Yes, utus increases on the left by s
Word repping Target State: sutus
Its inversion set is already represented in state_list
Status of targ_state_is_new: 
False
 
Word representing inversion set is: utus generator is: t
Yes, utus increases on the left by t
Word repping Target State: tutus
Its inversion set is already represented in state_list
Status of targ_state_is_new: 
False
 
Word representing inversion set is: utus generator is: u
No, utus decreases by u
 
Word representing inversion set is: utut generator is: s
Yes, utut increases on the left by s
Word repping Target State: sutut
Its inversion set is already represented in state_list
Status of targ_state_is_new: 
False
 
Word representing inversion set is: utut generator is: t
Yes, utut increases on the left by t
Word repping Target State: tutut
Status of targ_state_is_new: 
True
New State.

Status of targ_state_is_new: 
True
New State. Adding to state_list, the rep: tustutu
 
Word representing inversion set is: ustutu generator is: u
No, ustutu decreases by u
num_of_new_states: 2
NUMBER OF STATES IS NOW: 35
State reps are: 
['s', 't', 'u', 'ts', 'us', 'st', 'ut', 'tu', 'sts', 'uts', 'tus', 'ust', 'tut', 'stu', 'utu', 'usts', 'stus', 'utus', 'utut', 'ustu', 'tutu', 'tusts', 'ustus', 'tutut', 'tustu', 'stutu', 'ututu', 'utusts', 'tustus', 'stutut', 'ututut', 'stustu', 'ustutu', 'ustutut', 'tustutu']
The COUNTER is at: 33
 
Word representing inversion set is: ustutut generator is: s
No, ustutut decreases by s
 
Word representing inversion set is: ustutut generator is: t
Yes, ustutut increases on the left by t
Word repping Target State: tustutut
Status of targ_state_is_new: 
True
New State. Adding to state_list, the rep: tustutut
 
Word representing inversion set is: ustutut generator is: u
No, ustutut decreases by u
 
Word representing inversion set is: tustutu generator is:

Its inversion set is already represented in state_list
Status of targ_state_is_new: 
False
 
Word representing inversion set is: tustutustutut generator is: t
No, tustutustutut decreases by t
 
Word representing inversion set is: tustutustutut generator is: u
Yes, tustutustutut increases on the left by u
Word repping Target State: utustutustutut
Status of targ_state_is_new: 
True
New State. Adding to state_list, the rep: utustutustutut
num_of_new_states: 1
NUMBER OF STATES IS NOW: 46
State reps are: 
['s', 't', 'u', 'ts', 'us', 'st', 'ut', 'tu', 'sts', 'uts', 'tus', 'ust', 'tut', 'stu', 'utu', 'usts', 'stus', 'utus', 'utut', 'ustu', 'tutu', 'tusts', 'ustus', 'tutut', 'tustu', 'stutu', 'ututu', 'utusts', 'tustus', 'stutut', 'ututut', 'stustu', 'ustutu', 'ustutut', 'tustutu', 'tustutut', 'utustutu', 'utustutut', 'tutustutu', 'tutustutut', 'stutustutu', 'stutustutut', 'ututustutut', 'ustutustutut', 'tustutustutut', 'utustutustutut']
The COUNTER is at: 45
 
Word representing inversion set 

In [12]:
test

[[[(1.00000000000000, 0.000000000000000, 0.000000000000000)],
  ((0.000000000000000, 1.00000000000000, 0.000000000000000),
   [(0.000000000000000, 1.00000000000000, 0.000000000000000),
    (1.00000000000000, 1.00000000000000, 0.000000000000000)]),
  ((0.000000000000000, 0.000000000000000, 1.00000000000000),
   [(0.000000000000000, 0.000000000000000, 1.00000000000000),
    (1.00000000000000, 0.000000000000000, 0.000000000000000)])],
 [[(0.000000000000000, 1.00000000000000, 0.000000000000000)],
  ((1.00000000000000, 0.000000000000000, 0.000000000000000),
   [(1.00000000000000, 0.000000000000000, 0.000000000000000),
    (1.00000000000000, 1.00000000000000, 0.000000000000000)]),
  ((0.000000000000000, 0.000000000000000, 1.00000000000000),
   [(0.000000000000000, 0.000000000000000, 1.00000000000000),
    (0.000000000000000, 1.00000000000000, 1.00000000000000*sqrt(3))])],
 [[(0.000000000000000, 0.000000000000000, 1.00000000000000)],
  ((1.00000000000000, 0.000000000000000, 0.000000000000000)

In [193]:

def Hopcroft(states_transitions):
    '''Applies Hopcroft minimisation to bu-automata'''
    
    # list of states to be emptied
    list_of_states = []
    
    for states in states_transitions:
        list_of_states.append(states[0])
    
    # copy of list of states to initiate partition
    partition = [list_of_states[:]]
    
    while len(list_of_states) > 0:
        A = list_of_states[0]
        
        #remove from list of states
        list_of_states.remove(A)
        
        for i in range(0,dim_v):
            
            states_with_s_i_transition_to_A = []
            
            for st in states_transitions:
                for transition_info in st[1:]:
                    if transition_info[0] == s(i) and transition_info[1] == A:
                        states_with_s_i_transition_to_A.append(st)
            
            for y in partition:
                

16